# Visualization of the very lightweight DeepSDF network 

#### Therefore, ensure you have it downloaded and sampled points for objects are presented. If not, download the dataset and run sample_points.py script.

#### In this notebook I show a lightweight DeepSDF with size of each linear layer of 64. This net has more than 64 times less parameters than classical DeepSDF. Latent vector code has length of 32.

### Network's summary:

Decoder(
  
  (lin0): Linear(in_features=35, out_features=64, bias=True)
  
  (lin1): Linear(in_features=64, out_features=64, bias=True)
  
  (lin2): Linear(in_features=64, out_features=64, bias=True)
  
  (lin3): Linear(in_features=64, out_features=29, bias=True)
  
  (lin4): Linear(in_features=64, out_features=64, bias=True)
  
  (lin5): Linear(in_features=64, out_features=64, bias=True)
  
  (lin6): Linear(in_features=64, out_features=64, bias=True)
  
  (lin7): Linear(in_features=64, out_features=64, bias=True)
  
  (lin8): Linear(in_features=64, out_features=1, bias=True)
  
  (relu): ReLU()
  
  (th): Tanh()

)

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
import json
import torch
import trimesh
import logging
import numpy as np

from source.deep_sdf import Decoder
from source.deep_sdf_reconstructor import DeepSDFReconstructor
from source.utils import marching_cubes
from source.utils import download_models, data_loader

### Redirect trimesh log to the file for a clean cmd outputs

In [ ]:
log = logging.getLogger('trimesh')
fh = logging.FileHandler('trimesh_logs.log')
log.addHandler(fh)

### Set paths, urls and constants

In [ ]:
EPOCHS = (80, 130, 180, 230)
FACE_COLOR = (80, 255, 30)
train_id = 0
test_id = 0

source_url = r'https://www.dropbox.com/s/jivl308r3bzmmph/narrow_net.zip?dl=1'
target_dir = os.path.join(os.getcwd(), 'models')
target_file = os.path.join(target_dir, 'narrow_net.zip')

model_dir = os.path.join(target_dir, 'narrow_net')
specs_path = os.path.join(model_dir, 'specs.json')
split_path = os.path.join(model_dir, 'split.json')

### Download saved model weights and unpuck JSONs with metadata

In [ ]:
download_models(source_url, target_dir, target_file)

with open(specs_path, 'r') as f:
    specs = json.load(f)

with open(split_path, 'r') as f:
    split = json.load(f)

### Define more paths

In [ ]:
train_mesh_path = split['train'][train_id].split(os.sep)
test_mesh_path = split['test'][test_id].split(os.sep)
prefix = os.path.join(os.getcwd(), '..', 'data', 'ShapeNetCoreV2')

test_points_path = os.path.join(os.getcwd(), '..', *test_mesh_path)

train_mesh_path = os.path.join(prefix, *train_mesh_path[2:], 'models', 'model_normalized.obj')
test_mesh_path = os.path.join(prefix, *test_mesh_path[2:], 'models', 'model_normalized.obj')

### Calculate meshes for an object from training set

In [ ]:
data_gen = data_loader(path = model_dir, 
                       specs = specs, 
                       epochs = EPOCHS, 
                       obj_id = train_id)

In [ ]:
vert_list = []
face_list = []
i = 0
for net, latent in data_gen:
    verts, faces, normals, values = marching_cubes(net, latent, N = 128)
    verts[:, 0] += i # shift the object
    i += 2
    vert_list.append(verts)
    face_list.append(faces)

# Visualize a dynamics of the learning

In [ ]:

scene = trimesh.Scene()
for i, val in enumerate(zip(vert_list, face_list)):
    verts, faces = val
    
    mesh = trimesh.Trimesh(vertices=verts, faces=faces, face_colors=FACE_COLOR)
    scene.add_geometry(mesh)
scene.show()

# Compare a ground truth object with the one obtained from the network

In [ ]:
data_gen = data_loader(path = model_dir, 
                       specs = specs, 
                       epochs = EPOCHS[-1:], 
                       obj_id = train_id)

net, latent = [i for i in data_gen][0]
verts, faces, normals, values = marching_cubes(net, latent, N = 128)
verts[:, 1] -= 1
verts[:, 0] += 0.2

train_mesh = trimesh.Trimesh(vertices=verts, faces=faces, face_colors=FACE_COLOR)
scene.add_geometry(train_mesh)

gt_mesh = trimesh.load(train_mesh_path, force='mesh', skip_materials=True, face_colors=FACE_COLOR)

scene = trimesh.Scene()
scene.add_geometry(train_mesh)
scene.add_geometry(gt_mesh)
scene.show()

# Reconstruct an object from the test set

In [ ]:
data_gen = data_loader(path = model_dir, 
                       specs = specs, 
                       epochs = EPOCHS[-1:], 
                       obj_id = train_id)

net, latent = [i for i in data_gen][0]

latent_size = specs['CodeLength']
reconstrunctor = DeepSDFReconstructor(net, latent_size, num_iterations=200)
loss, latent_vec = reconstrunctor.reconstruct([test_points_path])

In [ ]:
verts, faces, normals, values = marching_cubes(net, latent_vec, N = 128)
verts[:, 1] -= 1
verts[:, 0] += 0.2

test_mesh = trimesh.Trimesh(vertices=verts, faces=faces, face_colors=FACE_COLOR)
scene.add_geometry(test_mesh)

gt_mesh = trimesh.load(test_mesh_path, force='mesh', skip_materials=True, face_colors=FACE_COLOR)

scene = trimesh.Scene()
scene.add_geometry(test_mesh)
scene.add_geometry(gt_mesh)
scene.show()